<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_Classification_12(%EC%99%80%EC%9D%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2유형

https://www.datamanim.com/dataset/03_dataq/typetwo.html#id20

## Classification (분류)
- datamanim 작업형에 regression 에 있지만 레드 와인 퀄리티 예측은 multi-classification

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [2]:
X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/y_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/x_test.csv")
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/y_test.csv')

display(X_train.head(3))
display(y_train.head(3))

display(X_test.head(3))
display(y_test.head(3))

,ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1,10.6,0.44,0.68,4.1,0.114,6.0,24.0,0.99700,3.06,0.66,13.4
1,2,7.0,0.60,0.30,4.5,0.068,20.0,110.0,0.99914,3.30,1.17,10.2
2,3,8.0,0.43,0.36,2.3,0.075,10.0,48.0,0.99760,3.34,0.46,9.4


,ID,quality
0,1,6
1,2,5
2,3,5


,ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,7.5,0.38,0.57,2.3,0.106,5.0,12.0,0.99605,3.36,0.55,11.4
1,7,8.2,0.24,0.34,5.1,0.062,8.0,22.0,0.99740,3.22,0.94,10.9
2,12,7.7,0.75,0.27,3.8,0.110,34.0,89.0,0.99664,3.24,0.45,9.3


,ID,quality
0,0,6
1,7,6
2,12,5


**`(1) RandomForestClassifier Base modeling으로 예측했을 때`**

In [4]:
X = X_train.drop(columns=['ID'])
y = y_train.drop(columns=['ID'])

display(X.head(3))
display(y.head(3))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,10.6,0.44,0.68,4.1,0.114,6.0,24.0,0.99700,3.06,0.66,13.4
1,7.0,0.60,0.30,4.5,0.068,20.0,110.0,0.99914,3.30,1.17,10.2
2,8.0,0.43,0.36,2.3,0.075,10.0,48.0,0.99760,3.34,0.46,9.4


,quality
0,6
1,5
2,5


(1)-1. Scaling 없이 모델링

In [5]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=2022)

print(X_train_m.shape)
print(y_train_m.shape)
print(X_test_m.shape)
print(y_test_m.shape)

(895, 11)
(895, 1)
(384, 11)
(384, 1)


In [7]:
model = RandomForestClassifier()
model.fit(X_train_m,y_train_m)
pred = model.predict_proba(X_test_m)
pred_class = model.predict(X_test_m)

accuracy_score(y_test_m, pred_class)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.6875

In [8]:
pred = model.predict_proba(X_test.drop(columns=['ID']))
pred_class = model.predict(X_test.drop(columns=['ID']))

accuracy_score(y_test['quality'], pred_class)

0.884375

(1)-2. Scaling 후 모델링

In [9]:
X_train.select_dtypes(include='number').columns

Index(['ID', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [10]:
from sklearn.preprocessing import StandardScaler

def Scaling(df):
  cols = df.select_dtypes(include='number').columns

  for col in cols:
    Scaler = StandardScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df

Scaling_X = Scaling(X)

display(Scaling_X.head(3))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1.295316,-0.497846,2.084690,1.139334,0.585295,-0.947299,-0.678485,0.147263,-1.591937,0.015500,2.750440
1,-0.748602,0.414110,0.145735,1.431359,-0.419351,0.402453,1.900273,1.288067,-0.062673,3.033524,-0.220215
2,-0.180847,-0.554843,0.451886,-0.174782,-0.266470,-0.561655,0.041169,0.467115,0.192204,-1.168039,-0.962879


In [12]:
model = RandomForestClassifier()
model.fit(Scaling_X,y)
pred = model.predict_proba(X_test.drop(columns=['ID']))
pred_class = model.predict(X_test.drop(columns=['ID']))

accuracy_score(y_test['quality'], pred_class)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.446875

-> 기본 RandomForset는 scaling 한 것보다 그냥 row data로 class를 예측하는게 더 높게 나옴
- scaling -> 0.44, row->0.88

## XGBboost + StratifiedKFold + RandomizedSearchCV 
( boosting 모델 + 교차검증 + 하이퍼파라미터 튜닝) 사용

In [14]:
from xgboost import XGBClassifier, XGBRegressor

In [15]:
display(X_train.head(3))
display(y_train.head(3))

display(X_test.head(3))
display(y_test.head(3))

,ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1,10.6,0.44,0.68,4.1,0.114,6.0,24.0,0.99700,3.06,0.66,13.4
1,2,7.0,0.60,0.30,4.5,0.068,20.0,110.0,0.99914,3.30,1.17,10.2
2,3,8.0,0.43,0.36,2.3,0.075,10.0,48.0,0.99760,3.34,0.46,9.4


,ID,quality
0,1,6
1,2,5
2,3,5


,ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,7.5,0.38,0.57,2.3,0.106,5.0,12.0,0.99605,3.36,0.55,11.4
1,7,8.2,0.24,0.34,5.1,0.062,8.0,22.0,0.99740,3.22,0.94,10.9
2,12,7.7,0.75,0.27,3.8,0.110,34.0,89.0,0.99664,3.24,0.45,9.3


,ID,quality
0,0,6
1,7,6
2,12,5


In [16]:
xgb_X_train = X_train.drop(columns=['ID'])
xgb_X_test = X_test.drop(columns=['ID'])

xgb_y_train = y_train.drop(columns=['ID'])
xgb_y_test = y_test.drop(columns=['ID'])

StratifiedKFold 사용 (교차검증)
- <아래 참조>
- https://continuous-development.tistory.com/166 

In [17]:
# 교차검증
skf = StratifiedKFold(n_splits=5, random_state=2023, shuffle=True)

# xgb model
xgb = XGBClassifier(random_state=42)
xgb_params = {'max_depth':[4,6,8], 'learning_rate':[0.05, 0.01, 0.1], 'n_estimators':[100,250]}

RandomizedSearchCV (하이퍼파라미터 튜닝)


In [18]:
# 하이퍼 파라미터 튜닝
rand_cv = RandomizedSearchCV(xgb, xgb_params, cv=skf, scoring='neg_log_loss', n_jobs=-1)

In [19]:
from sklearn.metrics import log_loss

In [20]:
rand_cv.fit(xgb_X_train, xgb_y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2023, shuffle=True),
                   estimator=XGBClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.01, 0.1],
                                        'max_depth': [4, 6, 8],
                                        'n_estimators': [100, 250]},
                   scoring='neg_log_loss')

In [21]:
rand_cv.best_params_

{'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100}

In [22]:
rand_cv.best_estimator_

XGBClassifier(learning_rate=0.05, max_depth=8, objective='multi:softprob',
              random_state=42)

In [23]:
xgb_pred = np.zeros((xgb_X_test.shape[0],6))
xgb_class = []

for train_idx, test_idx in skf.split(xgb_X_train, xgb_y_train):
  X_train_m, X_test_m = xgb_X_train.iloc[train_idx], xgb_X_train.iloc[test_idx]
  y_train_m, y_test_m = xgb_y_train.iloc[train_idx], xgb_y_train.iloc[test_idx]

  xgb = rand_cv.best_estimator_
  xgb.fit(X_train_m, y_train_m)

  val_pred = xgb.predict_proba(X_test_m)
  val_log_loss = log_loss(y_test_m, val_pred)
  print(val_log_loss)

  fold_pred = xgb.predict_proba(xgb_X_test)/5
  fold_class = xgb.predict(xgb_X_test)

  print(fold_class)

  xgb_pred += fold_pred
  xgb_class.append(fold_class)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8481076809330261
[6 6 5 3 4 6 5 6 5 5 5 5 5 6 6 6 6 5 6 5 6 6 5 6 6 6 5 6 5 5 5 6 6 6 5 6 5
 6 3 7 6 5 6 5 6 6 5 6 6 5 7 6 5 6 6 6 7 6 5 5 6 5 5 6 5 6 6 5 5 6 6 6 5 5
 7 6 6 7 6 6 5 7 6 7 6 6 6 5 5 4 5 6 5 5 6 5 6 5 5 5 6 5 6 5 5 6 7 7 5 5 5
 6 5 5 5 6 5 4 5 6 5 6 6 6 6 4 6 6 7 6 5 6 5 7 6 6 6 6 8 6 6 6 5 6 6 6 6 6
 7 5 5 6 6 6 5 8 5 5 5 5 6 5 6 6 6 5 5 5 5 6 6 6 5 6 6 6 5 5 5 5 6 5 5 6 6
 6 5 6 6 5 7 7 5 6 6 5 5 6 6 4 6 5 6 7 6 6 6 6 6 6 5 6 6 5 6 6 6 5 6 6 6 5
 5 6 6 6 7 6 7 5 5 5 6 5 5 5 5 5 5 6 6 5 5 5 6 5 5 6 6 4 6 4 5 6 5 5 5 5 4
 5 5 6 5 6 6 6 6 6 6 6 6 5 6 5 5 5 6 5 7 6 6 5 6 5 7 5 6 5 6 4 6 5 5 5 5 5
 8 5 6 6 6 6 5 5 6 5 6 5 6 6 7 6 5 6 6 5 6 5 5 7]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8229471832892159
[6 6 5 3 5 6 5 6 5 5 5 4 5 6 6 7 6 5 6 6 8 5 5 6 6 6 5 6 5 5 5 6 6 6 5 6 5
 6 5 7 6 5 6 5 6 6 5 5 6 5 7 5 5 6 6 5 7 6 5 5 6 5 5 6 5 6 6 5 5 6 6 6 5 5
 7 6 5 7 6 5 5 7 6 7 6 6 6 6 5 6 6 6 5 5 6 5 5 5 5 5 6 5 6 6 5 6 7 7 5 5 5
 6 5 6 5 6 5 4 5 6 5 6 6 6 6 5 6 6 7 6 5 5 5 7 6 6 6 6 8 6 6 5 5 6 5 6 6 6
 7 5 5 6 6 6 5 8 5 5 6 5 6 5 5 6 6 5 6 5 6 6 6 6 5 6 6 6 5 5 5 6 6 5 4 6 6
 6 5 6 6 5 7 7 5 6 6 5 5 6 6 7 6 5 6 7 7 6 7 6 6 6 5 5 6 5 6 6 6 6 6 6 6 5
 5 6 6 6 6 6 7 5 5 5 5 5 3 5 5 5 5 6 6 5 5 5 6 5 5 6 6 5 5 4 5 6 5 5 5 5 4
 5 5 6 5 6 6 6 6 4 6 6 6 5 6 6 5 5 5 5 7 6 6 5 5 5 6 5 5 5 6 3 6 5 5 5 5 5
 6 5 6 6 6 6 6 5 6 5 6 5 6 6 6 6 5 6 6 5 6 5 5 7]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9181052039348288
[6 6 5 3 5 6 5 6 5 5 5 4 5 6 6 7 6 5 6 5 8 6 5 6 6 6 5 6 5 5 5 6 6 6 5 6 5
 6 3 8 6 5 6 5 6 6 5 6 6 5 7 5 5 6 6 6 7 6 5 5 6 5 5 6 5 6 6 5 5 6 6 6 5 5
 7 6 6 7 6 5 5 7 6 6 6 6 6 7 5 6 5 6 5 5 6 5 5 5 5 6 6 5 6 5 5 6 8 7 5 5 5
 6 5 5 5 6 6 4 5 6 5 6 6 6 6 4 6 6 7 6 5 5 5 6 6 6 7 6 8 6 6 5 5 6 5 6 6 6
 7 5 5 6 6 6 5 8 5 5 5 5 6 5 6 6 6 5 6 5 5 6 6 6 5 6 5 6 6 5 5 6 6 5 4 6 6
 6 6 6 6 5 7 7 5 6 6 5 5 6 6 6 6 5 6 6 7 6 7 6 6 6 5 6 6 5 6 6 6 5 6 6 6 5
 5 6 6 6 6 5 7 5 5 5 6 6 3 5 5 5 5 6 6 5 5 6 6 5 5 6 6 5 5 6 5 6 5 6 5 5 5
 5 5 6 5 6 6 6 6 4 6 6 6 5 6 6 5 5 5 5 7 6 6 5 5 5 7 5 6 5 6 3 6 5 5 5 5 6
 6 5 6 7 6 6 6 5 6 5 6 5 6 6 7 5 5 6 6 5 6 5 5 7]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9486397692453465
[6 7 5 5 4 6 5 5 5 5 5 4 5 6 6 6 6 5 6 5 8 6 5 6 6 6 5 6 5 5 5 6 6 6 5 6 5
 6 3 7 6 5 6 5 6 6 5 6 6 5 7 5 5 6 6 6 7 6 5 5 6 5 5 6 5 6 6 5 5 6 7 6 5 5
 7 6 6 7 6 5 5 7 6 7 6 6 6 6 5 6 5 6 5 5 6 5 5 5 6 5 6 5 5 5 5 6 7 7 5 5 5
 5 5 5 5 5 6 4 5 6 5 6 6 6 5 4 6 6 7 6 5 5 5 7 6 6 6 6 7 6 6 5 5 6 6 6 6 6
 7 5 5 6 6 6 5 8 5 5 5 5 6 5 6 6 6 5 5 6 5 6 6 6 5 6 6 6 5 5 5 5 6 5 4 6 6
 6 5 6 6 5 6 7 5 6 6 5 5 6 5 6 5 5 6 7 7 5 7 6 6 6 5 5 6 5 6 6 6 5 6 6 6 5
 5 6 6 6 6 6 6 5 6 5 6 5 3 5 5 5 5 6 6 5 5 5 6 5 5 6 5 5 5 4 5 6 5 5 5 5 4
 5 5 6 5 6 5 6 6 4 6 7 6 5 6 5 5 5 5 5 7 6 7 5 5 6 6 5 5 5 6 3 6 5 5 5 5 5
 6 5 6 6 6 6 6 5 6 5 6 5 6 6 7 6 5 6 6 5 6 5 5 7]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9324935284023191
[6 6 5 3 5 6 5 6 5 5 5 4 5 6 6 7 6 5 6 5 8 6 6 6 6 6 5 5 6 5 5 6 6 5 5 6 5
 6 3 7 6 5 6 5 6 6 5 5 6 5 6 5 5 6 6 6 7 6 5 5 6 5 5 6 5 6 6 5 5 6 6 6 5 5
 5 5 6 7 6 5 5 7 6 7 6 6 6 5 5 6 5 6 6 5 6 5 5 5 5 5 6 5 6 5 5 6 7 7 5 5 5
 6 5 5 5 6 5 5 5 6 5 6 6 6 6 4 6 6 7 6 5 5 5 7 6 6 6 6 8 6 6 5 5 6 5 6 6 6
 6 5 6 6 6 6 5 7 5 5 5 5 6 5 6 7 6 5 6 5 5 6 6 6 5 6 6 6 5 5 5 5 6 5 4 6 6
 7 5 6 6 5 7 7 5 5 5 5 5 6 6 4 6 5 6 7 7 6 7 6 6 5 5 5 6 5 6 6 6 5 6 6 6 6
 5 6 6 6 6 6 7 5 5 6 6 5 3 6 5 5 5 6 6 5 5 5 6 5 5 6 6 5 5 4 5 6 5 5 5 5 4
 6 5 6 5 6 6 6 6 4 6 6 6 5 6 5 5 5 5 5 5 6 6 5 5 5 7 5 6 5 6 3 6 5 5 5 5 5
 6 5 6 6 6 6 5 5 6 5 6 5 6 6 7 6 5 6 6 5 6 5 5 7]


In [24]:
pred = np.argmax(xgb_pred, axis= 1)+3
pred

array([6, 6, 5, 3, 5, 6, 5, 6, 5, 5, 5, 4, 5, 6, 6, 7, 6, 5, 6, 5, 8, 6,
       5, 6, 6, 6, 5, 6, 5, 5, 5, 6, 6, 6, 5, 6, 5, 6, 3, 7, 6, 5, 6, 5,
       6, 6, 5, 6, 6, 5, 7, 5, 5, 6, 6, 6, 7, 6, 5, 5, 6, 5, 5, 6, 5, 6,
       6, 5, 5, 6, 6, 6, 5, 5, 7, 6, 6, 7, 6, 5, 5, 7, 6, 7, 6, 6, 6, 6,
       5, 6, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5, 6, 5, 5, 6, 7, 7, 5, 5,
       5, 6, 5, 5, 5, 6, 6, 4, 5, 6, 5, 6, 6, 6, 6, 4, 6, 6, 7, 6, 5, 5,
       5, 7, 6, 6, 6, 6, 8, 6, 6, 5, 5, 6, 5, 6, 6, 6, 7, 5, 5, 6, 6, 6,
       5, 8, 5, 5, 5, 5, 6, 5, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 5, 6, 6, 6,
       5, 5, 5, 5, 6, 5, 4, 6, 6, 6, 5, 6, 6, 5, 7, 7, 5, 6, 6, 5, 5, 6,
       6, 6, 6, 5, 6, 7, 7, 6, 7, 6, 6, 6, 5, 5, 6, 5, 6, 6, 6, 5, 6, 6,
       6, 5, 5, 6, 6, 6, 6, 6, 7, 5, 5, 5, 6, 5, 3, 5, 5, 5, 5, 6, 6, 5,
       5, 5, 6, 5, 5, 6, 6, 5, 5, 4, 5, 6, 5, 5, 5, 5, 4, 5, 5, 6, 5, 6,
       6, 6, 6, 4, 6, 6, 6, 5, 6, 5, 5, 5, 5, 5, 7, 6, 6, 5, 5, 5, 7, 5,
       5, 5, 6, 3, 6, 5, 5, 5, 5, 5, 6, 5, 6, 6, 6,

In [25]:
accuracy_score(xgb_y_test, np.argmax(xgb_pred, axis=1)+3)

0.915625

In [26]:
submission = pd.DataFrame({'ID':X_test['ID'],'pred' :pred, 'real':xgb_y_test['quality']})
submission

,ID,pred,real
0,0,6,6
1,7,6,6
2,12,5,5
3,13,3,3
4,25,5,6
...,...,...,...
315,1587,5,5
316,1589,6,6
317,1591,5,5
318,1592,5,5


-> XGBoosting model로 교차검증, 하이퍼파라미터 튜닝 시 0.91 까지 올릴 수 있음